In [1]:
# setup django project
import sys
sys.path.append("../../..")
import django
django.setup()

# general
import random
import json
import pandas as pd
from tqdm import tqdm

# django
from django.db import IntegrityError

# funcrowd
from tasks.models import (
    Mission,
    Task,
    ItemTemplate,
    Item,
    Annotation
)
from modules.order_strategy.models import Strategy
from modules.packages.models import MissionPackages, Package
from modules.feedback.models import Feedback, FeedbackScoreField


### 0. Setup default values in the database

In [2]:
Strategy.register_values()
FeedbackScoreField.register_values()

### 1. Create ItemTemplate from a json schema
1. Read a schema
2. Create a schema if it doesn't exist in the database yet

In [3]:
template_schema = json.load(open("./data/item_template_schema.json"))

In [4]:
try:
    template = ItemTemplate.create_template_from_schema(template_schema)
except IntegrityError as e:
    print(e)

duplicate key value violates unique constraint "tasks_itemtemplate_name_c8657b23_uniq"
DETAIL:  Key (name)=(example_task-ratio) already exists.



### 2. Create the Mission and Task objects

In [5]:
template = ItemTemplate.objects.get(name=template_schema['name'])
strategy = Strategy.objects.get(name="BreadthFirstStrategyLogic")

In [6]:
mission, _ = Mission.objects.get_or_create(
    name="exmple_mission - radio"
)

task1, _ = Task.objects.get_or_create(
    name="example_task - ratio - 1",
    mission=mission,
    strategy=strategy
)

feedback, created = Feedback.objects.get_or_create(task=task1)
if created:
    feedback.score_fields.add(FeedbackScoreField.objects.get(name="ReferenceScore"))

### 3. Create items and reference annotations

In [9]:
def _get_random_item_data():
    _first, _second = random.randint(1, 50), random.randint(1, 50)
    input_text = f"{_first} + {_second} = ?"
    correct_answer = _first + _second
    answers = [correct_answer]
    for _ in range(random.randint(2, 4)):
        _value = random.randint(1, 100)
        if _value not in answers:
            answers.append(_value)
    random.shuffle(answers)
    return input_text, correct_answer, answers


if task1.items.count() == 0:
    # create item and annotation with a correct answer
    for item_id in range(100):
        input_text, correct_answer, answers = _get_random_item_data()

        # create an item
        item = Item.objects.create(
            task=task1,
            template=template,
            data={
                "item_id": item_id,
                "input": input_text,
                "_output_sources": answers
            }
        )

        # create an annotation
        Annotation.objects.create(
            item=item,
            user=None,
            data={
                'output': correct_answer
            }
        )

### 4. Create packages

In [10]:
def create_packages(count, task, description, size=10):
    for _ in tqdm(range(count)):
        package = mp.create_package(
            size=size,
            task=task,
            name=f"{task1.name}.{mp.packages.count()}",
            metadata={
                "task": task.id,
                "description": description
            }
        )

In [11]:
mp, _ = MissionPackages.objects.get_or_create(
    mission=mission,
    max_annotations=10,
    strategy=strategy
)

#### 4.1. Create packages for the first task

In [14]:
if not task1.items.exclude(package=None).count():
    create_packages(10, task1, "high quality reference")